In [1]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt

data_folder = '../data/M-CAN Intrusion Dataset/'

def hex_to_bin(hex_num):
    
    binary_value = bin(int(str(hex_num), 16))[2:]
    
    return binary_value

def int_to_bin(int_num):
    
    binary_value = bin(int_num)[2:]
    
    return binary_value

hex_to_dec = lambda x: int(x, 16)
dec_to_hex = lambda x : hex(int(x))[2:]

In [2]:
def read_data(data_path):
    
    columns = ['Timestamp','ID', 'DLC', 'Payload', 'label']
    
    data = pd.read_csv(data_path)
    
    ##Replacing all NaNs with '00' 
    data = data.replace(np.NaN, '00')

    data['ID'] = data['ID'].apply(hex_to_dec)
    
    data['Payload'] = data['Payload'].str.replace(' ', '')
    data['Payload'] = data['Payload'].apply(hex_to_dec)
    
    # data = data.assign(IAT=data['Timestamp'].diff().fillna(0))
    
    return data

In [3]:
normal_data = read_data(os.path.join(data_folder, 'g80_mcan_normal_data.csv'))

In [4]:
normal_data = normal_data.iloc[:]

In [5]:
# normal_data.shape

In [6]:
## Function to calculate average time and frequency of each message ID
## In this function frequency is defined as the number of packets a particular message ID has
## Average time is defined as the average time between two packets of the same message ID
def calculate_average_time_frequency(dataframe):
    
    frame_times = defaultdict(list)
    total_frames = len(dataframe)
    id_frequency = dataframe["ID"].value_counts()
    id_frequency = defaultdict(int, id_frequency)

    for index, row in dataframe.iterrows():
        id_value = row["ID"]
        timestamp_value = row["Timestamp"]
        frame_times[id_value].append(timestamp_value)


    average_times = {}
    for arbitration_id, timestamps in frame_times.items():
        if len(timestamps) > 1:
            differences = [(timestamps[i+1] - timestamps[i]) for i in range(len(timestamps)-1)]
            average_times[arbitration_id] = sum(differences) / len(differences)
        else:
            average_times[arbitration_id] = 0

    return average_times, id_frequency, total_frames

In [7]:
def calculate_periodicity(df):
    # Initialize an empty dictionary to store the periodicity for each can_id
    periodicity_dict = {}

    # Group the DataFrame by 'can_id'
    grouped = df.groupby('ID')

    for can_id, group_df in grouped:
        # Sort the DataFrame by 'timestamp'
        sorted_df = group_df.sort_values(by='Timestamp')

        # Calculate the periodicity for the current can_id
        first_timestamp = sorted_df.iloc[0]['Timestamp']
        last_timestamp = sorted_df.iloc[-1]['Timestamp']
        num_occurrences = len(sorted_df)

        if num_occurrences > 1:
            periodicity = (last_timestamp - first_timestamp) / (num_occurrences - 1)
        else:
            periodicity = 0  # Avoid division by zero if there's only one occurrence

        # Store the periodicity in the dictionary
        periodicity_dict[can_id] = periodicity

    return periodicity_dict

def calculate_inverse_periodicity(periodicity_dict):
    
    inverse_periodicity_dict = {id: 1.0 / periodicity for id, periodicity in periodicity_dict.items()}
    
    return inverse_periodicity_dict

In [8]:
periodicity_dict = calculate_periodicity(normal_data)
print(periodicity_dict)

{133: 0.036633373046808804, 256: 1.9945216989843029, 290: 0.19995064292321924, 369: 0.19996919419002684, 371: 0.19998768668455633, 384: 0.9986221266759131, 385: 1.0000085555555556, 387: 0.2242656739356179, 466: 0.13579758811333795, 468: 0.2000073894132889, 722: 0.05000155054360398, 771: 0.20000614231516609, 772: 0.20000614231516609, 773: 0.20000614231516609, 774: 0.20000614231516609, 776: 0.20000619378123263, 778: 0.20000619378123263, 786: 0.20000618730334996, 787: 0.2000062011845271, 789: 0.2000060855080511, 792: 0.2000060855080511, 826: 0.20000611789746436, 829: 0.2000061913574535, 906: 0.20000737947626543, 1096: 0.2831337141359885, 1101: 0.2831359850668064, 1109: 0.2831360421852483, 1284: 0.20000620153604146, 1393: 1.000030217491902, 1400: 0.2000019311557324, 1408: 0.20000193300638477, 1409: 0.20000613491255667, 1410: 0.20000591838623116, 1411: 0.20000580457111133, 1414: 0.2000062024613676, 1425: 0.20000613861386138, 1427: 0.20000613861386138, 1430: 0.20000615341908023, 1435: 0.0500

In [9]:
# periodicity_dict = calculate_periodicity(normal_data)

# ## This is the time in seconds that the bus would take to transmit a packet which has 110 bits (maximum packet size)
# max_time_to_transmit = 0.00021

# periodicity_dict = calculate_periodicity(normal_data)
# sorted_periodicity_dict = dict(sorted(periodicity_dict.items(), key=lambda item: item[1], reverse=True))

# ## Retrieving message ID with highest periodicity, this will be the message ID that we will use to inject packets
# highest_periodicity_id = list(sorted_periodicity_dict.keys())[0]
# highest_periodicity =  list(sorted_periodicity_dict.values())[0]

# ## Retrieving message ID with lowest periodicity, our target message ID shouldnt have a lower periodicity than this after injection is complete
# lowest_periodicity_id = list(sorted_periodicity_dict.keys())[-1]
# lowest_periodicity =  list(sorted_periodicity_dict.values())[-1]

# ## Retrieving the indexes at which there is a message with highest periodicity message ID
# index_list_of_highest_arb_id = list(normal_data[normal_data['ID'] == highest_periodicity_id].index)

# ##TODO: currently only 1 packet is being inserted, choose frequency difference / lowest frequency
# ##TODO: Randomize attack packet arb id based on frequency : If injecting multiple packets at once, they can be of different IDs and follow the same attack principle

# out_data = []
# packets_inserted = 0
# pointer = 0
# offset = 0.00021 #make this dynamic depending on attack packet size


# # print(sorted_periodicity_dict)

# # print(highest_periodicity_id)

# # print(index_list_of_highest_arb_id)

In [10]:
# for index in tqdm(range(len(index_list_of_highest_arb_id) - 1)):

#     ts1 = normal_data['Timestamp'].iloc[index_list_of_highest_arb_id[index]]
#     ts2 = normal_data['Timestamp'].iloc[index_list_of_highest_arb_id[index + 1]]

#     # print("index_list_of_highest_arb_id[index]):",index_list_of_highest_arb_id[index])
#     # print("index_list_of_highest_arb_id[index+1]:",index_list_of_highest_arb_id[index+1])

#     last_encountered_ts = ts1

#     while last_encountered_ts < ts2:

#         # print(ts1, last_encountered_ts, ts2)  
#         ## DLC is always 8, data might not be

#         target_periodicity = 0.035 * lowest_periodicity

#         if target_periodicity <= max_time_to_transmit:
#             target_periodicity = lowest_periodicity
        
#         #Because we are trying to bring the periodcity of the attack IDs to the lowest
#         matching_ts = last_encountered_ts + target_periodicity

#         #Finding the location of where attack packet should be if we are trying to match the lowest periodicity
#         matching_packet_idx = normal_data[normal_data['Timestamp'] < matching_ts].index.max()
#         # print("matching_packet_idx:",matching_packet_idx)

#         #No injection packet can have an higher timestamp than this value
#         injection_ts_upper_bound = normal_data['Timestamp'].iloc[matching_packet_idx + 1] - max_time_to_transmit
#         interval = normal_data['Timestamp'].iloc[matching_packet_idx + 1] - normal_data['Timestamp'].iloc[matching_packet_idx]

#         # print(matching_packet_idx)
#         num_packets_to_insert = int((interval - max_time_to_transmit) // max_time_to_transmit)

#         # print("interval:", interval)

#         # print("num_packets_to_insert:",num_packets_to_insert)

#         # if counter != 0:
#         #     break

#         if num_packets_to_insert >= 1:

#             # Timestamp	ID	DLC	Payload	label
#             out_data.extend((normal_data.iloc[pointer:matching_packet_idx+1]).values)

#             attack_ts = normal_data['Timestamp'].iloc[matching_packet_idx] + target_periodicity
            
#             if attack_ts >= injection_ts_upper_bound:
#                 pointer = matching_packet_idx + 1
#                 last_encountered_ts = attack_ts
#                 break
#             else:
#                 attack_packet = [attack_ts, highest_periodicity_id, 8, np.random.choice(normal_data[normal_data['ID'] == highest_periodicity_id]['Payload']),1]
#                 out_data.append(attack_packet)
#                 packets_inserted += 1

#             for packets in range(1, num_packets_to_insert):
#                 rand_id = np.random.choice(list(sorted_periodicity_dict.keys())[:5])
#                 attack_ts += max_time_to_transmit + target_periodicity
                
#                 if attack_ts >= injection_ts_upper_bound:
#                     break
#                 else:
#                     attack_packet = [attack_ts, rand_id, 8,  np.random.choice(normal_data[normal_data['ID'] == rand_id]['Payload']),1]
#                     out_data.append(attack_packet)
#                     packets_inserted += 1
            

            

#             pointer = matching_packet_idx + 1
#             last_encountered_ts = attack_ts
        
#         else:
#             last_encountered_ts += target_periodicity 

        
# out_data.extend((normal_data.iloc[pointer:]).values)
# print(f"Packets inserted: {packets_inserted}")

In [11]:
periodicity_dict = calculate_periodicity(normal_data)

##Latest function
def add_dos(data, top_id_to_consider = 5):
    
    ## This is the time in seconds that the bus would take to transmit a packet which has 110 bits (maximum packet size)
    max_time_to_transmit = 0.00021
    
    periodicity_dict = calculate_periodicity(data)
    sorted_periodicity_dict = dict(sorted(periodicity_dict.items(), key=lambda item: item[1], reverse=True))

    ## Retrieving message ID with highest periodicity, this will be the message ID that we will use to inject packets
    highest_periodicity_id = list(sorted_periodicity_dict.keys())[0]
    highest_periodicity =  list(sorted_periodicity_dict.values())[0]

    ## Retrieving message ID with lowest periodicity, our target message ID shouldnt have a lower periodicity than this after injection is complete
    lowest_periodicity_id = list(sorted_periodicity_dict.keys())[-1]
    lowest_periodicity =  list(sorted_periodicity_dict.values())[-1]
    
    ## Retrieving the indexes at which there is a message with highest periodicity message ID
    index_list_of_highest_arb_id = list(data[data['ID'] == highest_periodicity_id].index)
    
    ##TODO: currently only 1 packet is being inserted, choose frequency difference / lowest frequency
    ##TODO: Randomize attack packet arb id based on frequency : If injecting multiple packets at once, they can be of different IDs and follow the same attack principle
    

    out_data = []
    packets_inserted = 0
    pointer = 0 
    offset = 0.00021 #make this dynamic depending on attack packet size
    
    for index in tqdm(range(len(index_list_of_highest_arb_id) - 1)):

        ts1 = normal_data['Timestamp'].iloc[index_list_of_highest_arb_id[index]]
        ts2 = normal_data['Timestamp'].iloc[index_list_of_highest_arb_id[index + 1]]

        # print("index_list_of_highest_arb_id[index]):",index_list_of_highest_arb_id[index])
        # print("index_list_of_highest_arb_id[index+1]:",index_list_of_highest_arb_id[index+1])

        last_encountered_ts = ts1

        while last_encountered_ts < ts2:

            # print(ts1, last_encountered_ts, ts2)  
            ## DLC is always 8, data might not be

            ##Adjust this parameter for attack intensity
            target_periodicity = 0.026 * lowest_periodicity

            if target_periodicity <= max_time_to_transmit:
                target_periodicity = lowest_periodicity
                print("Forced to change target periodicity to lowest periodicity")
            
            #Because we are trying to bring the periodcity of the attack IDs to the lowest
            matching_ts = last_encountered_ts + target_periodicity

            #Finding the location of where attack packet should be if we are trying to match the lowest periodicity
            matching_packet_idx = normal_data[normal_data['Timestamp'] < matching_ts].index.max()
            # print("matching_packet_idx:",matching_packet_idx)

            #No injection packet can have an higher timestamp than this value
            injection_ts_upper_bound = normal_data['Timestamp'].iloc[matching_packet_idx + 1] - max_time_to_transmit
            interval = normal_data['Timestamp'].iloc[matching_packet_idx + 1] - normal_data['Timestamp'].iloc[matching_packet_idx]

            # print(matching_packet_idx)
            num_packets_to_insert = int((interval - max_time_to_transmit) // max_time_to_transmit)

            # print("interval:", interval)

            # print("num_packets_to_insert:",num_packets_to_insert)

            # if counter != 0:
            #     break

            if num_packets_to_insert >= 1:

                # Timestamp	ID	DLC	Payload	label
                out_data.extend((normal_data.iloc[pointer:matching_packet_idx+1]).values)

                attack_ts = normal_data['Timestamp'].iloc[matching_packet_idx] + target_periodicity
                
                if attack_ts >= injection_ts_upper_bound:
                    pointer = matching_packet_idx + 1
                    last_encountered_ts = attack_ts
                    break
                else:
                    attack_packet = [attack_ts, highest_periodicity_id, 8, np.random.choice(normal_data[normal_data['ID'] == highest_periodicity_id]['Payload']),1]
                    out_data.append(attack_packet)
                    packets_inserted += 1

                for packets in range(1, num_packets_to_insert):
                    rand_id = np.random.choice(list(sorted_periodicity_dict.keys())[:5])
                    attack_ts += max_time_to_transmit + target_periodicity
                    
                    if attack_ts >= injection_ts_upper_bound:
                        break
                    else:
                        attack_packet = [attack_ts, rand_id, 8,  np.random.choice(normal_data[normal_data['ID'] == rand_id]['Payload']),1]
                        out_data.append(attack_packet)
                        packets_inserted += 1
                

                

                pointer = matching_packet_idx + 1
                last_encountered_ts = attack_ts
            
            else:
                last_encountered_ts += target_periodicity 

            
    out_data.extend((normal_data.iloc[pointer:]).values)
    print(f"Packets inserted: {packets_inserted}")
    return out_data

In [ ]:
out = add_dos(normal_data)
out_df = pd.DataFrame(out, columns = ['Timestamp','ID', 'DLC', 'Payload', 'label'])

 37%|███████████████████████████████████████▎                                                                 | 405/1083 [15:50<1:15:41,  6.70s/it]

In [ ]:
out_df = out_df.assign(IAT=out_df['Timestamp'].diff().fillna(0))

In [ ]:
attack_periodicity = calculate_periodicity(out_df)

In [ ]:
arb_ids = list(periodicity_dict.keys())

benign_periodicity_values = [periodicity_dict[can_id] for can_id in arb_ids]
attack_periodicity_values = [attack_periodicity[can_id] for can_id in arb_ids]

periodicity_df = pd.DataFrame(columns = 
                              ['ID', 'Benign Periodicity', 'Attack Periodicity'],
                              data = {'ID': arb_ids,
                                      'Benign Periodicity': benign_periodicity_values,
                                      'Attack Periodicity': attack_periodicity_values})

periodicity_df['Delta'] = abs(periodicity_df['Benign Periodicity'] - periodicity_df['Attack Periodicity'])

periodicity_df = periodicity_df.sort_values(by='Delta', ascending=False)

periodicity_df['ID'] = periodicity_df['ID'].apply(dec_to_hex)

periodicity_df.head()

In [ ]:
# Plotting
fig, ax = plt.subplots(figsize=(20, 6))

bar_width = 0.35
bar_positions = range(len(periodicity_df))

benign_bars = ax.bar(bar_positions, periodicity_df['Benign Periodicity'], bar_width, label='Periodicity in Benign Data')
smart_bars = ax.bar([p + bar_width for p in bar_positions], periodicity_df['Attack Periodicity'], bar_width, label='Periodicity after Adversarial Attack')

ax.set_xticks([p + bar_width / 2 for p in bar_positions])
ax.set_xticklabels(periodicity_df['ID'])

ax.set_xlabel('ID')
ax.set_ylabel('Periodicity')
ax.set_title('Benign vs Attack Periodicity')
ax.legend()

plt.show()
